![walmartecomm](walmartecomm.jpg)

Walmart is the biggest retail store in the United States. Just like others, they have been expanding their e-commerce part of the business. By the end of 2022, e-commerce represented a roaring $80 billion in sales, which is 13% of total sales of Walmart. One of the main factors that affects their sales is public holidays, like the Super Bowl, Labour Day, Thanksgiving, and Christmas. 

In this project, you have been tasked with creating a data pipeline for the analysis of supply and demand around the holidays, along with conducting a preliminary analysis of the data. You will be working with two data sources: grocery sales and complementary data. You have been provided with the `grocery_sales` table in `PostgreSQL` database with the following features:

# `grocery_sales`
- `"index"` - unique ID of the row
- `"Store_ID"` - the store number
- `"Date"` - the week of sales
- `"Weekly_Sales"` - sales for the given store

Also, you have the `extra_data.parquet` file that contains complementary data:

# `extra_data.parquet`
- `"IsHoliday"` - Whether the week contains a public holiday - 1 if yes, 0 if no.
- `"Temperature"` - Temperature on the day of sale
- `"Fuel_Price"` - Cost of fuel in the region
- `"CPI"` – Prevailing consumer price index
- `"Unemployment"` - The prevailing unemployment rate
- `"MarkDown1"`, `"MarkDown2"`, `"MarkDown3"`, `"MarkDown4"` - number of promotional markdowns
- `"Dept"` - Department Number in each store
- `"Size"` - size of the store
- `"Type"` - type of the store (depends on `Size` column)

You will need to merge those files and perform some data manipulations. The transformed DataFrame can then be stored as the `clean_data` variable containing the following columns:
- `"Store_ID"`
- `"Month"`
- `"Dept"`
- `"IsHoliday"`
- `"Weekly_Sales"`
- `"CPI"`
- "`"Unemployment"`"

After merging and cleaning the data, you will have to analyze monthly sales of Walmart and store the results of your analysis as the `agg_data` variable that should look like:

|  Month | Weekly_Sales  | 
|---|---|
| 1.0  |  33174.178494 |
|  2.0 |  34333.326579 |
|  ... | ...  |  

Finally, you should save the `clean_data` and `agg_data` as the csv files.

It is recommended to use `pandas` for this project. 

In [ ]:
import pandas as pd
grocery_sales = pd.read_csv('grocery_sales.csv')

In [ ]:
import pandas as pd
import os

# Extract function is already implemented for you 
def extract(store_data, extra_data):
    extra_df = pd.read_parquet(extra_data)
    merged_df = store_data.merge(extra_df, on = "index")
    return merged_df

# Call the extract() function and store it as the "merged_df" variable
merged_df = extract(grocery_sales, "extra_data.parquet")
merged_df.columns

In [ ]:
# Create the transform() function with one parameter: "raw_data"
def transform(raw_data):
    columns = raw_data.columns[raw_data.isna().any()][1:] 
    clean_data = raw_data.fillna(raw_data[columns].mean())
    clean_data['Date'] = pd.to_datetime(clean_data['Date'])
    clean_data['Month'] = clean_data['Date'].dt.month
    clean_data = clean_data.loc[clean_data['Weekly_Sales']>10000, :]
    clean_data  = clean_data.drop(columns = ['Date','index', 'Temperature', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3','MarkDown4', 'MarkDown5', 'Type', 'Size'])
    return clean_data

In [ ]:
# Call the transform() function and pass the merged DataFrame
clean_data = transform(merged_df)


In [ ]:
def avg_weekly_sales_per_month(clean_data):
    temp_df = clean_data[['Weekly_Sales', "Month"]].copy()
    agg_data = temp_df.groupby('Month').mean().reset_index().round(2)
    agg_data = agg_data.rename(columns={'Weekly_Sales': 'Avg_Sales'})
    return agg_data

In [ ]:
result = avg_weekly_sales_per_month(clean_data) 

In [ ]:
# Create the load() function that takes in the cleaned DataFrame and the aggregated one with the paths where they are going to be stored
def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    full_data.to_csv(full_data_file_path, index = False)
    agg_data.to_csv(agg_data_file_path, index = False)

In [ ]:
load(clean_data, 'clean_data.csv', result, 'agg_data.csv') 

In [ ]:
def validation(file_path):
    return os.path.exists(file_path)

In [ ]:
print(validation('clean_data.csv'))
print(validation('agg_data.csv'))